In [161]:
%pip install numpy
%pip install sklearn

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


In [162]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

In [105]:
data = pd.read_csv('/home/jupyter/mnt/s3/newbacketttt/train_new.csv', encoding = 'utf-8')

train_X = data['sentence'].values
train_y = data['sentiment'].values

X_train, X_test, y_train, y_test = train_test_split(train_X, train_y, test_size=0.1, random_state=42)

In [106]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [107]:
knn = KNeighborsClassifier(n_neighbors=7)

clf = knn.fit(X_train_tfidf, y_train)

X_new_counts = count_vect.transform(X_test)

X_new_tfidf = tfidf_transformer.transform(X_new_counts)

In [108]:
predicted = clf.predict_proba(X_new_tfidf)

In [109]:
from sklearn.metrics import roc_auc_score

In [100]:
test_roc_auc = roc_auc_score(y_test, predicted, multi_class='ovr', labels=['+', '?', '−'])

In [101]:
test_roc_auc

0.8948662698893557

In [110]:
test_roc_auc = roc_auc_score(y_test, predicted, multi_class='ovr', labels=[0, 1, 2])

In [111]:
test_roc_auc

0.8829015873309239

# Second version

In [ ]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(X_train)

# Catboost

In [168]:
%pip install catboost

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


In [169]:
from catboost import CatBoostClassifier
from catboost import Pool

In [170]:
data = pd.read_csv('/home/jupyter/mnt/s3/newbacketttt/train_new.csv', encoding = 'utf-8')

train_X = data[['sentence']]
train_y = data['sentiment']

X_train, X_test, y_train, y_test = train_test_split(train_X, train_y, test_size=0.1, random_state=42)

In [171]:
train_pool = Pool(
    X_train, 
    y_train, 
    text_features=['sentence'], 
)
valid_pool = Pool(
    X_test, 
    y_test,
    text_features=['sentence'], 
)

catboost_params = {
    'iterations': 400,
    'learning_rate': 0.01,
    'eval_metric': 'AUC',
    'task_type': 'CPU',
    'early_stopping_rounds': 100,
    'use_best_model': True,
    'verbose': 50
}
# + - 0
# -- - 2  
# ? - 1
clas_weight = [0.3, 0.15, 0.52]
model = CatBoostClassifier(**catboost_params,class_weights=clas_weight)
model.fit(train_pool, eval_set=valid_pool)

0:	test: 0.9231181	best: 0.9231181 (0)	total: 291ms	remaining: 1m 55s
50:	test: 0.9398255	best: 0.9401145 (45)	total: 11.3s	remaining: 1m 17s
100:	test: 0.9418158	best: 0.9420081 (95)	total: 22.5s	remaining: 1m 6s
150:	test: 0.9419105	best: 0.9420081 (95)	total: 34s	remaining: 56s
200:	test: 0.9421202	best: 0.9421334 (193)	total: 45.2s	remaining: 44.7s
250:	test: 0.9426912	best: 0.9426912 (250)	total: 56.2s	remaining: 33.4s
300:	test: 0.9429296	best: 0.9430687 (277)	total: 1m 8s	remaining: 22.4s
350:	test: 0.9432875	best: 0.9433343 (345)	total: 1m 19s	remaining: 11s
399:	test: 0.9436261	best: 0.9436466 (398)	total: 1m 29s	remaining: 0us

bestTest = 0.9436465709
bestIteration = 398

Shrink model to first 399 iterations.


In [172]:
probas = model.predict_proba(X_test)

In [173]:
probas

array([[0.01749256, 0.02164604, 0.96086141],
       [0.01577648, 0.01952048, 0.96470304],
       [0.84729424, 0.07274904, 0.07995672],
       ...,
       [0.01751624, 0.02098309, 0.96150066],
       [0.01769221, 0.02189143, 0.96041636],
       [0.92328089, 0.03363226, 0.04308686]])

In [164]:
test_roc_auc = roc_auc_score(y_test, probas, multi_class='ovr', labels=[0, 1, 2])
test_roc_auc

0.9279262063428805

In [174]:
%pip install pymystem3

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


In [175]:
import torch
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from tqdm import tqdm

In [176]:
%pip install natasha

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


In [177]:
import string
import pandas as pd
import torch
import natasha
from razdel import tokenize, sentenize
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from string import digits
from pymystem3 import Mystem
mystem = Mystem()

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jupyter/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [178]:
ds = pd.read_csv('/home/jupyter/mnt/s3/newbacketttt/1sentencenewtest.csv')

In [180]:
df = ds['0'].copy()

0      15.03.2022 обратился на горячую линию для закр...
1      Уже который год в ТКБ не решается "глобальная ...
2                                            Добрый день
3      Добрый день Сегодня, зайдя в свой личный кабин...
4      Обслуживаюсь в Тинькофф пару лет, возникла жес...
                             ...                        
944    Отвратительный сервис и отношение к клиентам! ...
945    28.04.2022 обратилась в банк о возможности пер...
946    В начале 2021 года была акция по выплате 8% ке...
947     Бездействие банка и некомпетентность сотрудников
948    Потрачено 5 часов чтобы произвести оплату за о...
Name: 0, Length: 949, dtype: object